In [1]:
import pandas as pd

df = pd.read_csv('../data/sample_products.csv')

df.head()

,product_id,name,category,brand,price,tags,description
0,1,PageTurner Electronics #1,Electronics,PageTurner,439.23,battery|wireless|bluetooth|4K,PageTurner electronics item designed for batte...
1,2,PageTurner Beauty #2,Beauty,PageTurner,292.59,anti-aging|SPF|vitamin-c|sensitive|skincare,PageTurner beauty item designed for anti-aging...
2,3,PageTurner Fashion #3,Fashion,PageTurner,373.81,summer|formal|casual|leather,PageTurner fashion item designed for summer an...
3,4,Silk&Co Home #4,Home,Silk&Co,402.76,eco-friendly|durable|minimal,Silk&Co home item designed for eco-friendly an...
4,5,PageTurner Beauty #5,Beauty,PageTurner,337.18,SPF|hydrating|anti-aging|skincare|vitamin-c,PageTurner beauty item designed for SPF and hy...


In [2]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['category'] = encoder.fit_transform(df['category'])

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'product_name' is a text feature
vectorizer = TfidfVectorizer(max_features=100)
X = vectorizer.fit_transform(df['name'])


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

scaler = StandardScaler()

df['price_scaled'] = scaler.fit_transform(df[['price']])

df['category_price_interaction'] = df['category'] * df['price_scaled']


description_vectorizer = TfidfVectorizer(max_features=100)
description_features = description_vectorizer.fit_transform(df['description'])

mlb = MultiLabelBinarizer()
tags_features = mlb.fit_transform(df['tags'])

In [17]:
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Encode categorical feature using LabelEncoder
encoder = LabelEncoder()
df['category'] = encoder.fit_transform(df['category'])

# Apply TF-IDF to product names
vectorizer = TfidfVectorizer(max_features=100)
product_name_features = vectorizer.fit_transform(df['product_name'])

# Normalize price
scaler = StandardScaler()
df['price_scaled'] = scaler.fit_transform(df[['price']])

# TF-IDF for product descriptions
description_vectorizer = TfidfVectorizer(max_features=100)
description_features = description_vectorizer.fit_transform(df['description'])

# Multi-label binarization for tags
mlb = MultiLabelBinarizer()
tags_features = mlb.fit_transform(df['tags'])

# Feature interaction (category * price)
df['category_price_interaction'] = df['category'] * df['price_scaled']

# One-Hot Encoding for category (alternative to Label Encoding)
encoder = OneHotEncoder(sparse=False)
category_encoded = encoder.fit_transform(df[['category']])
category_encoded_df = pd.DataFrame(category_encoded, columns=encoder.categories_[0])
df = pd.concat([df, category_encoded_df], axis=1)

# User interaction aggregation (if applicable)
user_interaction_agg = df.groupby('user_id')['interactions'].agg(['mean', 'sum', 'count']).reset_index()
df = pd.merge(df, user_interaction_agg, on='user_id', how='left')

# Feature importance using a RandomForest (example)
X = df.drop(columns=['target'])
y = df['target']
model = RandomForestClassifier()
model.fit(X, y)
feature_importances = model.feature_importances_

# Save the engineered dataset
df.to_csv('engineered_features.csv', index=False)


KeyError: 'product_name'

In [18]:
print(df.columns.tolist())


['product_id', 'name', 'category', 'brand', 'price', 'tags', 'description', 'price_scaled', 'category_price_interaction', 'scaled_price']
